# 📌8-4. 텍스트 분류 실습 - 20 뉴스그룹 분류

텍스트 분류 : 특정 문서의 분류를 학습 데이터를 통해 학습해 모델을 생성한 뒤 이 모델로 다른 문서의 분류를 예측!  
  
사이킷런 내부의 데이터인 20 뉴스그룹 데이터로 예제 실습.  
텍스트를 피처 벡터화로 변환하면 일반적으로 희소 행렬 형태가 되는데,  
이러한 희소 행렬에 분류를 잘 처리하는 알고리즘은 **로지스틱 회귀, SVM, 나이브 베이즈** 등이 있다.  
  
본 예제에선, 아래의 과제들을 수행함.
- 카운트 기반과 TF-IDF의 벡터화 성능을 비교
- 피처 벡터화를 위한 파라미터와 GridSearchCV 기반의 하이퍼 파라미터 튜닝
- 사이킷런의 Pipeline 객체를 통해 파라미터 튜닝을 한꺼번에 수행


## 1. 텍스트 정규화

In [1]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all',random_state=156)

In [2]:
#key값 확인
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


In [3]:
import pandas as pd
#Target 클래스 구성 살펴보기
print('target 클래스의 값과 분포도 \n',pd.Series(news_data.target).value_counts().sort_index())
print('target 클래스의 이름들 \n',news_data.target_names)

target 클래스의 값과 분포도 
 0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스의 이름들 
 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [4]:
#데이터 한개만 출력해보기
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

위의 결과를 보면 기사 내용뿐만 아닌 뉴스그룹 제목, 작성자, 이메일 등의 정보 또한 포함하고 있는 것을 확인 가능.  
이 중 내용을 제외하곤 모두 제거. 왜냐하면 그 외의 애들(제목이나 소속 등)은 타겟 클래스와 유사한 데이터를 가진 경우가 많기에 이들을 넣으면 당연히 정확도가 올라가기 때문.  
따라서 순수한 텍스트로만 구성된 기사 내용으로 분류를 진행하기로 함.  
remove 파라미터를 이용하여 푸터와 헤더 등을 모두 제거.

In [5]:
from sklearn.datasets import fetch_20newsgroups

#subset = 'train'으로 학습용 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
train_news = fetch_20newsgroups(subset = 'train', remove=('headers', 'footers', 'quotes'), random_state=156)
X_train = train_news.data
y_train = train_news.target
print(type(X_train))

#subset='test'으로 테스트 데이터만 추출, remove=('headers', 'footers', 'quotes')로 내용만 추출
test_news = fetch_20newsgroups(subset = 'test',remove=('headers', 'footers','quotes'),random_state=156)
X_test = test_news.data
y_test = test_news.target
print('학습 데이터 크기 {0} , 테스트 데이터 크기 {1}'.format(len(train_news.data) , len(test_news.data)))

<class 'list'>
학습 데이터 크기 11314 , 테스트 데이터 크기 7532


## 2. 피처 벡터화 변환과 머신러닝 모델 학습/예측/평가

피처 벡터화 시에 유의할 점!  
 : 반드시 학습 데이터를 이용해 fit한 뒤에 테스트 데이터를 변환해야함!  
*(흠 근데 원래 다른 애들도 이렇게 했던것 같다... 내가 기억이 틀린가.. 그렇게 배웠는디...)*  
왜냐하면 CountVectorizer를 사용할 때 그렇게 하지 않으면 학습데이터와 테스트데이터의 피처 개수가 달라지기 때문!!

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

#Count Vectorization으로 피처 벡터화 변환 수행.
cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

#학습 데이터로 fit()된 CountVectorizer를 이용해 테스트 데이터를 피처 벡터화 변환 수행.
X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape:', X_train_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape: (11314, 101631)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

#LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_cnt_vect , y_train)
pred = lr_clf.predict(X_test_cnt_vect)
print('CountVectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test,pred)))

CountVectorized Logistic Regression 의 예측 정확도는 0.617


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF 벡터화를 적용해 학습 데이터 세트와 테스트 데이터 세트 변환.
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

#LogisticRegression을 이용하여 학습/예측/평가 수행.
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.678


TF-IDF의 정확도가 단순 카운트 기반보다 더 높은 것을 확인할 수 있음.  
일반적으로 문서 내에 텍스트가 많고 많은 문서를 가지는 텍스트 분석에선 TF-IDF가 좋은 성능을 보임!

위에선 기본 파라미터만 적용했지만 보다 더 성능을 향상시키기 위해서 파라미터를 다양하게 적용해보기로 함.

In [9]:
#stop words 필터링을 추가하고 ngram을 기본(1,1)에서 (1,2)로 변경해 피처 벡터화 적용.
tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.690


In [10]:
from sklearn.model_selection import GridSearchCV

#최적 C값 도출 튜닝 수행. CV는 3 폴드 세트로 설정.
params = {'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter :',grid_cv_lr.best_params_ )

#최적 C값으로 학습된 grid_cv로 예측 및 정확도 평가.
pred = grid_cv_lr.predict(X_test_tfidf_vect)
print('TF-IDF Vectorized Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression의 예측 정확도는 0.704


## 3. 사이킷런 파이프라인 사용 및 GridSearchCV와의 결합

In [ ]:
#있길래 적긴 했지만 뒤에서도 새로 객체 정의?해서... 파이프라인 불러오지도 않았을 타이밍이라서 각주 처리해뒀습니다!
#pipeline = Pipeline([('tfidf_vect', TfidfVectorizer(stop_words='english')),
#    ('lr_clf', LogisticRegression(random_state = 156))])

In [11]:
from sklearn.pipeline import Pipeline

#TfidfVectorizer 객체를 tfidf_vect로, LogisticRegression 객체를 lr_clr로 생성하는 Pipeline 생성
pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(solver='liblinear', C=10))
])

#별도의 TfidfVectorizer객체의 fit(), transform()과 LogisticRegression의 fit(), predict( )가 필요 없음.
#pipeline의 fit() 과 predict()만으로 한꺼번에 피처 벡터화와 ML 학습/예측이 가능.
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
print('Pipeline을 통한 Logistic Regression의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Pipeline을 통한 Logistic Regression의 예측 정확도는 0.704


In [12]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

#Pipeline에 기술된 각각의 객체 변수에 언더바(_)2개를 연달아 붙여 GridSearchCV에 사용될
#파라미터/하이퍼 파라미터 이름과 값을 설정.
params = {'tfidf_vect__ngram_range': [(1,1), (1,2), (1,3)],
          'tfidf_vect__max_df': [100, 300, 700],
          'lr_clf__C': [1, 5, 10]
}

#GridSearchCV의 생성자에 Estimator가 아닌 Pipeline 객체 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid = params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train , y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
print('Pipeline을 통한 Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(y_test ,pred)))

Fitting 3 folds for each of 27 candidates, totalling 81 fits


KeyboardInterrupt: ignored

3시간까지 기다렸지만 결과가 나오지 않아서 중단시켰습니다.